# Kaggle - TMDB 

### Extract tables - production_companies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import math

In [2]:
MILLION = 1000000

In [3]:
train = pd.read_csv('train.csv')

### production_companies

In [4]:
train['production_companies'].fillna('[]').map(lambda x: len(ast.literal_eval(x))).value_counts()

1     775
2     734
3     582
4     312
5     166
0     156
6     118
7      62
8      42
9      29
11      7
10      7
12      3
16      2
15      2
14      1
13      1
17      1
Name: production_companies, dtype: int64

In [20]:
train['prod'] = train['production_companies'].fillna('[{"id":-123}]').map(lambda x: [i['id'] for i in (ast.literal_eval(x))])

In [21]:
productions = []
for p in train[train['production_companies'].notnull()]['production_companies'].map(lambda x: [(i['id'],i['name']) for i in (ast.literal_eval(x))]).values:
    productions.extend(p)

prod_id = [p[0] for p in productions]
prod_name = [p[1] for p in productions]

productions,counts = np.unique(np.array(prod_id),return_counts=True)

prod_names = {}
for i in range(len(prod_id)):
    prod_names[prod_id[i]] = prod_name[i]

In [22]:
for i in range(20):
    print(i,len([prod_names[p] for p in productions[counts==i]]))
print('>20',len([prod_names[p] for p in productions[counts>50]]))

0 0
1 2678
2 505
3 188
4 105
5 60
6 38
7 28
8 18
9 16
10 8
11 9
12 4
13 4
14 4
15 2
16 4
17 1
18 3
19 2
>20 11


Try only productions that have at least 5 movies

In [37]:
revenues = {}
for i in prod_names.keys():
    if (counts[productions==i]>5):
        revenues[i] = train[train['prod'].map(lambda x: True if i in x else False)]['revenue'].map(math.log).mean()

In [41]:
revenues

{4: 17.381079061980937,
 60: 17.000036930747143,
 8411: 16.413752160032484,
 2: 18.857807128309688,
 3172: 18.064468560641856,
 97: 17.37482457838468,
 306: 17.662572621798066,
 10201: 17.31646346120715,
 27: 18.480664455113253,
 215: 16.890685472749276,
 216: 17.58664945475536,
 56: 18.579960510083907,
 6363: 16.840137128242116,
 288: 15.45742696753967,
 2452: 16.689118611850255,
 3823: 17.686016241423342,
 5358: 15.685160602050402,
 6896: 17.100827639075387,
 12: 17.90916624265092,
 33: 17.69511779837755,
 552: 16.63077061231688,
 9: 15.984250889393502,
 6301: 17.747526527292003,
 5870: 16.173375127711036,
 6916: 13.25406166330179,
 7025: 14.441710289819904,
 491: 17.318067347756102,
 7437: 15.582464378113974,
 6194: 17.528503145345162,
 441: 16.870350274797268,
 126: 16.54091571502828,
 12026: 18.15296692336638,
 9383: 19.853968763510917,
 11749: 19.691829752138194,
 308: 16.587559287758072,
 559: 16.675636728961102,
 5: 18.050588153838948,
 915: 16.700645164769917,
 1382: 17.175570

In [44]:
paol = pd.Series(revenues).to_frame()

In [45]:
paol.columns = ['revenue']
paol.head()

,revenue
4,17.381079
60,17.000037
8411,16.413752
2,18.857807
3172,18.064469


In [46]:
paol['counts'] = paol.index.map(dict(zip(productions,counts)))
paol.head()

,revenue,counts
4,17.381079,161
60,17.000037,44
8411,16.413752,84
2,18.857807,62
3172,18.064469,18


In [47]:
paol['name'] = paol.index.map(prod_names)
paol.head()                  

,revenue,counts,name
4,17.381079,161,Paramount Pictures
60,17.000037,44,United Artists
8411,16.413752,84,Metro-Goldwyn-Mayer (MGM)
2,18.857807,62,Walt Disney Pictures
3172,18.064469,18,Blumhouse Productions


In [48]:
paol.to_csv('production_companies_short.csv')